In [18]:
%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [1]:
from __future__ import division

import sys
import time
import logging

import numpy as np
from scipy.io import loadmat
from os import path

# change this to the path of mushu if you don't have it in your
# PYTHONPATH already
sys.path.append('../../mushu')
sys.path.append('../')

import libmushu
from libmushu.driver.replayamp import ReplayAmp
from wyrm.types import RingBuffer
import wyrm.processing as proc
from wyrm import io
from wyrm.types import Data, BlockBuffer, RingBuffer

#Classifiers
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import matplotlib
import matplotlib.pyplot as plt

DEBUG:matplotlib:$HOME=C:\Users\aleja
DEBUG:matplotlib:matplotlib data path C:\Users\aleja\Anaconda3\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:loaded rc file C:\Users\aleja\Anaconda3\lib\site-packages\matplotlib\mpl-data\matplotlibrc
DEBUG:matplotlib:matplotlib version 2.2.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'errno', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprl

DEBUG:matplotlib:CACHEDIR=C:\Users\aleja\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\aleja\.matplotlib\fontList.json
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
logging.basicConfig(format='%(relativeCreated)10.0f %(threadName)-10s %(name)-10s %(levelname)8s %(message)s', level=logging.NOTSET)
logger = logging.getLogger(__name__)

In [3]:
# replay the experiment in real time?
REALTIME = False


TRAIN_DATA = '../BCI_Comp_III_Wads_2004/data/Subject_A_Train.mat'
TEST_DATA = '../BCI_Comp_III_Wads_2004/data/Subject_A_Test.mat'

CHANNEL_DATA = '../BCI_Comp_III_Wads_2004/data/eloc64.txt'

TRUE_LABELS_TEST = "WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU"

STIMULUS_CODE = {
    # cols from left to right
    1 : "agmsy5",
    2 : "bhntz6",
    3 : "ciou17",
    4 : "djpv28",
    5 : "ekqw39",
    6 : "flrx4_",
    # rows from top to bottom
    7 : "abcdef",
    8 : "ghijkl",
    9 : "mnopqr",
    10: "stuvwx",
    11: "yz1234",
    12: "56789_"
}

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {i : [i] for i in STIMULUS_CODE.values()}

SEG_IVAL = [0, 800]

In [4]:
def load_bci_TESTdata(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal'][60:,:,:]
    data = data.astype('double')
#     print('data: ',data.shape)
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'][60:,:]
    flashing = flashing.reshape(-1)
#     print('flashing: ',flashing.shape)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'][60:,:].reshape(-1)
#     print('stim_code: ', stimulus_code.shape)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([]))[60:,:].reshape(-1)
#     print('stim_type: ', stimulus_type.shape)
    # The target characters
    target_chars = data_m.get('TargetChar')[0][60:]
#     target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [5]:
def load_bci_TRAINdata(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal'][:60,:,:]
    data = data.astype('double')
#     print('data: ',data.shape)
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'][:60,:]
    flashing = flashing.reshape(-1)
#     print('flashing: ',flashing.shape)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'][:60,:].reshape(-1)
#     print('stim_code: ', stimulus_code.shape)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([]))[:60,:].reshape(-1)
#     print('stim_type: ', stimulus_type.shape)
    # The target characters
    target_chars = data_m.get('TargetChar')[0][:60]
#     target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [6]:
def train(filename):
    cnt = load_bci_TRAINdata(filename)

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

#     b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
#     cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 20)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.create_feature_vectors(epo)
    X = fv.data
    y = fv.axes[0]
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, train_size = 0.7)

    #     clf = proc.lda_train(fv)
    clf = LinearDiscriminantAnalysis(solver='eigen')
    clf.fit(X,y)
    return clf

In [108]:
markers_processed = 12
correct_letter = False

if (markers_processed%12 == 0) & (markers_processed != 180) & (not correct_letter):   
    print('Hlas')

Hlas


In [204]:
def online_experiment(amp, cfy):
    amp_fs = amp.get_sampling_frequency()
    amp_channels = amp.get_channels()
    
    file = open('sim_training_LDA.txt', 'w')
    file.write('LDA Online simulation')


    #buf = BlockBuffer(4)
    rb = RingBuffer(5000)

    fn = amp_fs / 2
    b_low, a_low = proc.signal.butter(5, [10 / fn], btype='low')
#     b_high, a_high = proc.signal.butter(5, [.4 / fn], btype='high')

    zi_low = proc.lfilter_zi(b_low, a_low, len(amp_channels))
#     zi_high = proc.lfilter_zi(b_high, a_high, len(amp_channels))

    amp.start()
    markers_processed = 0
#     current_letter_idx = 0
#     current_letter = TRUE_LABELS[current_letter_idx].lower()

    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
    endresult = []
    file.write('\nCharacter #'+str(len(endresult)+1))

   # t0 = time.time()
    #data, markers = amp.get_data()
    
    y_true = np.array([])
    y_pred = np.array([])
    
    correct_letter = False #boolean to know if the letter has already been predicted
    
    while True:
        t0 = time.time()

        # get fresh data from the amp
        data, markers = amp.get_data()
        if len(data) == 0:
            continue

        # we should rather wait for a specific end-of-experiment marker
        if len(data) == 0:
            break

        # convert to cnt
        cnt = io.convert_mushu_data(data, markers, amp_fs, amp_channels)
        
        ## enter the block buffer
        #buf.append(cnt)
        #cnt = buf.get()
        #if not cnt:
        #    continue

        # low-pass and subsample
        cnt, zi_low = proc.lfilter(cnt, b_low, a_low, zi=zi_low)
#         cnt, zi_high = proc.lfilter(cnt, b_high, a_high, zi=zi_high)

        cnt = proc.subsample(cnt, 20)

        newsamples = cnt.data.shape[0]

        # enter the ringbuffer
        rb.append(cnt)
        cnt = rb.get()

        # segment
        epo = proc.segment_dat(cnt, MARKER_DEF_TEST, SEG_IVAL, newsamples=newsamples)
        epo_train = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL, newsamples=newsamples)
        if not epo: # If not enough data for 800 ms
            continue

        fv = proc.create_feature_vectors(epo)
        fv_train = proc.create_feature_vectors(epo_train)
        
        logger.debug('Markers processed: ')
        logger.debug(markers_processed)
        
        lda_out = cfy.predict_proba(fv.data)[:,1]
        y_true = np.append(y_true, fv_train.axes[0])
        
        y_pred_now = cfy.predict(fv.data) 
        y_pred = np.append(y_pred, y_pred_now)
        
        y_correct = y_pred_now == fv_train.axes[0]
        print('Predicho: ', fv_train.axes[0],'\tVerdadero: ', y_pred_now, y_correct )

#         lda_out = proc.lda_apply(fv, cfy)
        markers = [fv.class_names[cls_idx] for cls_idx in fv.axes[0]]
        result = zip(markers, lda_out)
        
        report = metrics.classification_report(y_true, y_pred)
        recall = metrics.recall_score(y_true, y_pred)
        precision = metrics.precision_score(y_true, y_pred)
        accuracy = metrics.accuracy_score(y_true, y_pred)
        f1 = metrics.f1_score(y_true, y_pred)
        print('\nP300 classification metrics...')
        print('recall: ', recall, '\tprecision: ', precision, '\tf1: ', f1, '\taccuracy: ', accuracy)
        for s, score in result: #letters in one row/column
          
            for letter in s:
                letter_prob[letter] += score
            if (markers_processed%12 == 0):
                trues = '\n\nTrial #' + str(int(markers_processed/12)+1) + ':\t' + str(np.count_nonzero(y_pred[-12:] == y_true[-12:])) + '/12 correct P300 predictions'
                file.write(trues)
                res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
                file.write('\nAccumulated metrics: ')
                file.write(res)
                current_letter = sorted(letter_prob.items(), key=lambda x: x[1])[-1][0]
                if (current_letter.upper() == TRUE_LABELS[len(endresult)]) & (not correct_letter):
                    file.write('\n\n~~~~~Character #{} ({}) predicted in {} trials'.format(len(endresult)+1,
                                                                                  current_letter, int(markers_processed/12)+1))
                    correct_letter = True
#                 if (markers_processed != 180) & (not correct_letter):
#                     if(current_letter.upper() == TRUE_LABELS[len(endresult)]):
#                          file.write('\n\nCharacter #{} ({}) predicted in {} trials'.format(len(endresult),
#                                                                                            current_letter, int(markers_processed/12)))
                if markers_processed == 180: # 12 trials x 15 repetitions = 180
                    endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
                    # Set all probabilities to zero
                    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
                    markers_processed = -1
                    if not correct_letter:
                        file.write('\n\nFailed to predict character #{}'.format(len(endresult)))
    #                         file.write('\nPredicted {} instead of {}.'.format(current_letter.upper(), TRUE_LABELS[len(endresult)]))
    #                         res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
    #                         file.write(res)

                    correct_letter = False
                    file.write('\n\nCharacter #'+str(len(endresult)+1))
        

            markers_processed += 1

                
#             if (markers_processed%12 == 0) & (markers_processed != 180) & (not correct_letter):
#                 current_letter = sorted(letter_prob.items(), key=lambda x: x[1])[-1][0]
#                 if(current_letter.upper() == TRUE_LABELS[len(endresult)]):
#                     file.write('\n\nCharacter #{} ({}) predicted in {} trials'.format(len(endresult),
#                                                                                   current_letter, int(markers_processed/12)))
#                     res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
#                     file.write(res)
#                     correct_letter = True
           
#             elif markers_processed == 180: # 12 trials x 15 repetitions = 180
#                 endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
#                 # Set all probabilities to zero
#                 letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
#                 markers_processed = 0
#                 if not correct_letter:
#                     file.write('\n\nFailed to predict character #{} in 15 trials'.format(len(endresult)))
#                     file.write('\nPredicted {} instead of {}.'.format(current_letter.upper(), TRUE_LABELS[len(endresult)]))
#                     res = '\nrecall: ' + str(recall) + '\tprecision: ' + str(precision) + '\tf1: ' + str(f1) + '\taccuracy: ' + str(accuracy)
#                     file.write(res)
                    
#                 correct_letter = False
# #                 current_letter_idx += 1
# #                 print('letter idx', current_letter_idx)
# #                 current_letter = TRUE_LABELS[current_letter_idx].lower()
#             markers_processed += 1
#         print(letter_prob)
#         logger.debug("".join([i[0] for i in sorted(letter_prob.items(), key=lambda x: x[1], reverse=True)]).replace(current_letter, " %s " % current_letter))

        # calculate the current accuracy
        if len(endresult) > 0:
            logger.debug(TRUE_LABELS)
            logger.debug("".join(endresult))
            acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
            print('Character prediction metrics...')
            print("Current accuracy:", acc * 100)
            print('{}/{} characters predicted'.format(len(endresult), len(TRUE_LABELS)))
        if len(endresult) == len(TRUE_LABELS)-1:
            break
        #logger.debug("Result: %s" % result)
        print('time',1000 * (time.time() - t0))

    acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
    print ("Accuracy:", acc * 100)
    file.close()

    amp.stop()


In [8]:
data_m = loadmat(TRAIN_DATA)
target_chars = data_m.get('TargetChar')[0][60:]
TRUE_LABELS = target_chars

In [9]:
TRUE_LABELS[1]

'_'

In [10]:
TRUE_LABELS

'W_EKTLBWXEPOUIKZERYOOTHQI'

# 1. Training (off-line)

In [11]:
clf = train(TRAIN_DATA)

# 2. Online experiment

In [12]:
bbuffer = BlockBuffer(12)
cnt = load_bci_TESTdata(TRAIN_DATA)

In [13]:
cnt.names

['time', 'channel']

In [205]:
amp = RReeplayAmp()
amp.configure(data=cnt.data, marker=cnt.markers, channels=cnt.axes[-1], fs=cnt.fs,
              realtime=False, blocksize_samples=12)

In [206]:
online_experiment(amp, clf)

DEBUG:__main__:Markers processed: 
DEBUG:__main__:0
DEBUG:__main__:Markers processed: 
DEBUG:__main__:1
DEBUG:__main__:Markers processed: 
DEBUG:__main__:2
DEBUG:__main__:Markers processed: 
DEBUG:__main__:3
DEBUG:__main__:Markers processed: 
DEBUG:__main__:4
DEBUG:__main__:Markers processed: 
DEBUG:__main__:5
DEBUG:__main__:Markers processed: 
DEBUG:__main__:6
DEBUG:__main__:Markers processed: 
DEBUG:__main__:7
DEBUG:__main__:Markers processed: 
DEBUG:__main__:8
DEBUG:__main__:Markers processed: 
DEBUG:__main__:9


Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 6.942987442016602
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 6.944417953491211
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 7.9364776611328125
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 8.430957794189453
Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  1.0 	precision:  1.0 	f1:  1.0 	accuracy:  1.0
time 11.903047561645508
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  1.0 	precision:  1.0 	f1:  1.0 	accuracy:  1.0
time 11.408329010009766
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  1.0 	preci

DEBUG:__main__:Markers processed: 
DEBUG:__main__:10
DEBUG:__main__:Markers processed: 
DEBUG:__main__:11
DEBUG:__main__:Markers processed: 
DEBUG:__main__:12
DEBUG:__main__:Markers processed: 
DEBUG:__main__:13
DEBUG:__main__:Markers processed: 
DEBUG:__main__:14
DEBUG:__main__:Markers processed: 
DEBUG:__main__:15
DEBUG:__main__:Markers processed: 
DEBUG:__main__:16
DEBUG:__main__:Markers processed: 
DEBUG:__main__:17


Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.5 	precision:  0.5 	f1:  0.5 	accuracy:  0.8181818181818182
time 14.387130737304688
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5 	precision:  0.5 	f1:  0.5 	accuracy:  0.8333333333333334
time 17.85421371459961
Predicho:  [1] 	Verdadero:  [1] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8461538461538461
time 9.423255920410156
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8571428571428571
time 10.911703109741211
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8666666666666667
time 12.40086555480957
Predicho:  [1] 	Verdadero:  [1] [

DEBUG:__main__:Markers processed: 
DEBUG:__main__:18
DEBUG:__main__:Markers processed: 
DEBUG:__main__:19
DEBUG:__main__:Markers processed: 
DEBUG:__main__:20
DEBUG:__main__:Markers processed: 
DEBUG:__main__:21
DEBUG:__main__:Markers processed: 
DEBUG:__main__:22


...

Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5623268698060941 	precision:  0.36609558160504957 	f1:  0.4434735117422174 	accuracy:  0.7646651270207853
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 20.830869674682617
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.5623268698060941 	precision:  0.36576576576576575 	f1:  0.44323144104803486 	accuracy:  0.7644885707688756
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 21.823644638061523
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.5623268698060941 	precision:  0.36543654365436545 	f1:  0.4429896344789962 	accuracy:  0.7643120960295475
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 22.321224212646484
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
rec

DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:172
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:173
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:174
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:175
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:176



P300 classification metrics...
recall:  0.5623268698060941 	precision:  0.36543654365436545 	f1:  0.4429896344789962 	accuracy:  0.764475201845444
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 23.312091827392578
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.5623268698060941 	precision:  0.36510791366906475 	f1:  0.44274809160305345 	accuracy:  0.7642988929889298
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 22.815942764282227
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5623268698060941 	precision:  0.36510791366906475 	f1:  0.44274809160305345 	accuracy:  0.7643532395665207
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 22.319793701171875
Predicho:  [1] 	Verdadero:  [0] [False]

P300 classification metrics...
recall:  0.5615491009681881 	precision:  0.

DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:177
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:178
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:179
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootb
DEBUG:__main__:Markers processed: 
DEBUG:__main__:180
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouijterzootbq



recall:  0.5621546961325967 	precision:  0.3653500897666068 	f1:  0.4428726877040261 	accuracy:  0.7640552995391705
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 25.792360305786133
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5621546961325967 	precision:  0.3653500897666068 	f1:  0.4428726877040261 	accuracy:  0.7641096521538816
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 22.321462631225586
Predicho:  [0] 	Verdadero:  [1] [False]

P300 classification metrics...
recall:  0.5621546961325967 	precision:  0.3650224215246637 	f1:  0.4426318651441001 	accuracy:  0.7639336711192999
Character prediction metrics...
Current accuracy: 78.26086956521739
23/25 characters predicted
time 22.81665802001953
Predicho:  [0] 	Verdadero:  [0] [ True]

P300 classification metrics...
recall:  0.5621546961325967 	precision:  0.3650224215246637 	f1:  0.44263186514

In [15]:


from __future__ import division

import time

import numpy as np

from libmushu.amplifier import Amplifier


class RReeplayAmp(Amplifier):

    def configure(self, data, marker, channels, fs, realtime=True, blocksize_ms=None, blocksize_samples=None):
        """

        Parameters
        ----------
        data
        marker
        channels
        fs
        realtime
        blocksize_ms : float
            blocksize in milliseconds
        blocksize_samples : int
            blocksize in samples

        Raises
        ------
        TypeError :
            if ``blocksize_ms`` and ``blocksize_s`` are given.

        """
        if [blocksize_ms, blocksize_samples].count(None) != 1:
            raise TypeError("blocksize_ms and blocksize_samples are mutually exclusive.")

        self.data = data
        # slow python
        self.marker = marker
        # fast numpy
        self.marker_ts = np.array([ts for ts, s in marker])
        self.marker_s = np.array([s for ts, s in marker])
        self.channels = channels
        self.fs = fs
        self.realtime = realtime

        if blocksize_ms:
            samples = fs * (blocksize_ms / 1000)
            if not samples.is_integer():
                raise ValueError("Resulting blocksize is not integer, please fix the blocksize_ms.")
            self.samples = samples
        if blocksize_samples:
            self.samples = blocksize_samples

    def start(self):
        self.last_sample_time = time.time()
        self.pos = 0

    def stop(self):
        pass

    def get_data(self):
        """

        Returns
        -------
        chunk, markers: Markers is time in ms since relative to the
        first sample of that block.

        """
        if self.realtime:
            elapsed = time.time() - self.last_sample_time
            blocks = (self.fs * elapsed) // self.samples
            samples = int(blocks * self.samples)
            
        else:
            samples = self.samples
      
        elapsed = samples / self.fs
        self.last_sample_time += elapsed
        # data
        chunk = self.data[self.pos:int(self.pos+samples)]
        #self.data = self.data[samples:]
        # markers

        # slow python version
        #markers = [x for x in self.marker if x[0] < elapsed * 1000]
        #self.marker = [x for x in self.marker if x[0] >= elapsed * 1000]
        #self.marker = [[x[0] - elapsed * 1000, x[1]] for x in self.marker]

        # fast numpy version
        mask = self.marker_ts < (elapsed * 1000) 
        markers = list(zip(self.marker_ts[mask], self.marker_s[mask]))
        self.marker_ts = self.marker_ts[~mask]
        self.marker_s = self.marker_s[~mask]
        self.marker_ts -= elapsed * 1000

        self.pos += samples
        return chunk, markers

    def get_channels(self):
        return self.channels

    def get_sampling_frequency(self):
        return self.fs

    @staticmethod
    def is_available():
        return True


In [156]:
a=np.arange(13)
b=np.arange(13)
b[10]=0

In [160]:
np.count_nonzero(a[-12:] == b[-12:])

11